In [9]:
%%writefile app.py

import spotipy
from spotipy.oauth2 import SpotifyOAuth
import streamlit as st
from datetime import datetime, timedelta

# Set up Spotify API credentials
client_id = "enter_client_id_from_spotify_developer_portal"
client_secret = "enter_client_secret_from_spotify_developer_portal"
redirect_uri = "enter_redirect_uri_as_in_spotify_developer_portal"
scope = "user-read-recently-played user-top-read"

# Create a Spotify client
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope))

# Initialize session state
if 'recommendations' not in st.session_state:
    # Step 1: Get the user's recently played tracks from the past 3 months
    three_months_ago = datetime.now() - timedelta(days=90)
    results = sp.current_user_recently_played(limit=50)  # Adjust the limit as needed
    recent_tracks = [item["track"] for item in results["items"] if datetime.strptime(item["played_at"], "%Y-%m-%dT%H:%M:%S.%fZ") >= three_months_ago]

    # Step 2: Get recommendations based on the recent tracks (limited to 5 seed tracks)
    track_ids = [track["id"] for track in recent_tracks[:5]]  # Limit to a maximum of 5 seed tracks
    recommendations = sp.recommendations(seed_tracks=track_ids, limit=5)

    tracks = recommendations["tracks"]

    recommendation_track_ids = [track["id"] for track in tracks]

    # Calculate average values for energy and instrumentalness
    audio_features = sp.audio_features(tracks=recommendation_track_ids)
    energy_sum = sum(track["energy"] for track in audio_features)
    instrumentalness_sum = sum(track["instrumentalness"] for track in audio_features)
    num_tracks = len(tracks)

    avg_energy = energy_sum / num_tracks if num_tracks > 0 else 0
    avg_instrumentalness = instrumentalness_sum / num_tracks if num_tracks > 0 else 0

    st.session_state.recommendations = recommendations
    st.session_state.avg_energy = avg_energy
    st.session_state.avg_instrumentalness = avg_instrumentalness
    st.session_state.track_ids = track_ids

# Streamlit app
st.title("Spotify Recommendations")

# Display the number of tracks considered
st.write(f"Considering {len(st.session_state.track_ids)} tracks played in the past 3 months")

# Create sliders for energy and instrumentalness
energy = st.slider("Energy", min_value=0.0, max_value=1.0, value=st.session_state.avg_energy, step=0.05)
instrumentalness = st.slider("Instrumentalness", min_value=0.0, max_value=1.0, value=st.session_state.avg_instrumentalness, step=0.05)

# Function to display track information with thumbnail
def display_track(track, index):
    col1, col2 = st.columns([1, 3])
    with col1:
        st.image(track['album']['images'][0]['url'], width=100)
    with col2:
        st.write(f"{index}. {track['name']} by {track['artists'][0]['name']}")
        st.write(f"Album: {track['album']['name']}")

# Button to get new recommendations
if st.button("Get New Recommendations"):
    new_recommendations = sp.recommendations(seed_tracks=st.session_state.track_ids, limit=5,
                                             target_energy=energy,
                                             target_instrumentalness=instrumentalness)
    st.session_state.recommendations = new_recommendations

# Display the current recommendations
for i, track in enumerate(st.session_state.recommendations["tracks"], start=1):
    display_track(track, i)

Overwriting app.py
